In [1]:
import pandas as pd

# Clean and preprocess the data

### Get the data in one single dataframe

In [2]:
def read_csv_files(destination, dates, result=None):
    """
    This function concatenates multiple CSV files into a single DataFrame. 
    If the result is None, it creates a list of DataFrames from the CSV files and concatenates them. 
    If the result is not None, it appends the DataFrames from the CSV files to the result.

    Args:
    -----
    result : The DataFrame to append the data to. If None, a new DataFrame is created.
 
    dates (list): The list of dates to use for the filenames of the CSV files.
    
    destination (str): The destination to use for the filenames of the CSV files.
 
    Return:
    -------
    DataFrame: The concatenated DataFrame.
    """
    if result is None:
        dfs = []
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_{destination}\\booking_{date}.csv"
            df = pd.read_csv(filename)
            dfs.append(df)
        result = pd.concat(dfs, axis=0, ignore_index=True)
    else:
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_{destination}\\booking_{date}.csv"
            df = pd.read_csv(filename)
            result = pd.concat([result, df], axis=0, ignore_index=True)
    return result

In [3]:
def transfomed_df1(df):
    """
    This function transforms a DataFrame by splitting and renaming columns, 
    converting data types, and applying functions to columns.

    Args:
    -----
    df : The DataFrame to transform.
 
    Return:
    -------
    DataFrame: The transformed DataFrame.
    """
    
    df['airline_company'] = df['airline_company'].str.split(",").str[0]
    df['out_stop_num'] = df['out_stop_num'].str.split(' ').str[0]
    df['in_stop_num'] = df['in_stop_num'].str.split(' ').str[0]

    split_df = df.pop('price_ticket').str.rsplit(' ', n=1, expand=True).rename(columns={0: 'ticket_price', 1: 'currency'})
    df = df.join(split_df)
    df['ticket_price'] = df['ticket_price'].str.replace(',', '.').str.replace(' ', '').astype(float)

    date_cols = [col for col in df.columns if col.endswith('_date')]
    year= '2023'
    for col in date_cols:
        df[col] = pd.to_datetime(df[col] + ' ' + year, format='%b %d %Y')
    
    time_cols = [col for col in df.columns if col.endswith('_time')]
    df[time_cols] = df[time_cols].apply(lambda x: pd.to_datetime(x, format='%I:%M %p').dt.strftime('%H:%M'))

    duration_cols = [col for col in df.columns if col.endswith('_duration')]
    df[duration_cols] = df[duration_cols].applymap(lambda x: pd.to_timedelta(x.replace('h', ' hours ').replace('m', ' min')))

    return df

In [4]:
def transfomed_df2(df):
    """
    This function transforms a DataFrame by splitting and renaming columns, 
    converting data types, and applying functions to columns.

    Args:
    -----
    df : The DataFrame to transform.
 
    Return:
    -------
    DataFrame: The transformed DataFrame.
    """
    air_cols = [col for col in df.columns if col.endswith('_airline_company')]
    df[air_cols] = df[air_cols].astype(str)
    df[air_cols] = df[air_cols].apply(lambda x: x.str.split(",").str[0])
    
    df['out_stop_num'] = df['out_stop_num'].str.split(' ').str[0]
    df['in_stop_num'] = df['in_stop_num'].str.split(' ').str[0]

    split_df = df.pop('price_ticket').str.rsplit(' ', n=1, expand=True).rename(columns={0: 'ticket_price', 1: 'currency'})
    df = df.join(split_df)
    df['ticket_price'] = df['ticket_price'].str.replace(',', '.').str.replace(' ', '').astype(float)

    date_cols = [col for col in df.columns if col.endswith('_date')]
    year= '2023'
    for col in date_cols:
        df[col] = pd.to_datetime(df[col] + ' ' + year, format='%b %d %Y')
    
    time_cols = [col for col in df.columns if col.endswith('_time')]
    df[time_cols] = df[time_cols].apply(lambda x: pd.to_datetime(x, format='%I:%M %p').dt.strftime('%H:%M'))

    duration_cols = [col for col in df.columns if col.endswith('_duration')]
    df[duration_cols] = df[duration_cols].applymap(lambda x: pd.to_timedelta(x.replace('h', ' hours ').replace('m', ' min')))

    return df


## 1. Jakarta

#### Initial dataframe

This dataframe has some missing values such as airline company for the outbound and inbound flights. 
We have observed that some round-trip flights have 2 different airline companies that we did not take into account. This initial dataframe will not be taken account. 

In [5]:
dates = ["14_may_06_PM", "15_may_07_AM", "15_may_10_AM", "15_may_01_PM", "15_may_04_PM", "15_may_07_PM"]
dates.extend(["16_may_07_AM", "16_may_10_AM", "16_may_01_PM", "16_may_04_PM", "16_may_07_PM"])
dates.extend(["17_may_07_AM", "17_may_10_AM"])

In [6]:
# Initial dataframe without in_airline_company and out_airline_company
df1 = read_csv_files( "Jakarta", dates)

In [7]:
df1 = transfomed_df1(df1)

#### Second Dataframe

This dataframe has taken into account airline companies for the outbound and inbound flights. 
This dataframe will be the final dataframe that we will use.

In [8]:
dates_final = ["20_may_09_AM", "20_may_12_PM"]


In [9]:
df_jakarta = read_csv_files( "Jakarta", dates_final)
df_jakarta = transfomed_df2(df_jakarta)

In [10]:
# Create a boolean mask to identify rows with different outbound and inbound airlines
mask = df_jakarta['out_airline_company'] != df_jakarta['in_airline_company']

# Use the mask to get the rows with different outbound and inbound airlines
result = df_jakarta[mask]

In [11]:
df_jakarta = df_jakarta.loc[df_jakarta['out_airline_company'] == df_jakarta['in_airline_company']]

In [12]:
df_jakarta = df_jakarta.drop_duplicates()
df_jakarta = df_jakarta.reset_index(drop=True)

## 2. Tokyo

The daframe for Tokyo will not be taken into account for the simple reason that there is not enough data coming from Qatar Airways. 

In [13]:
dates = ["14_may_06_PM", "15_may_07_AM"]
df3 = read_csv_files( "Tokyo", dates)

In [14]:
df3 = df3.loc[df3['arr_city'] != 'CGK']
df3 = df3.reset_index(drop=True)

In [15]:
dates = ["15_may_10_AM", "15_may_01_PM", "15_may_04_PM", "15_may_07_PM"]
dates.extend(["16_may_07_AM","16_may_10_AM","16_may_01_PM","16_may_04_PM", "16_may_07_PM"])
dates.extend(["17_may_07_AM", "17_may_10_AM"])

In [16]:
df4 = read_csv_files( "Tokyo", dates)
df3 = pd.concat([df3, df4], ignore_index=True)

In [17]:
Tokyo_cleaned_data= transfomed_df1(df3)

In [18]:
dates_3 = ["17_may_02_PM"]
df5 = read_csv_files( "Tokyo", dates_3)

In [19]:
df5 = transfomed_df2(df5)
dif = df5['out_airline_company'] != df5['in_airline_company']
result = df5[dif]

In [20]:
qatar_rows = df5[(df5['out_airline_company'] == 'Qatar Airways') & (df5['in_airline_company'] == 'Qatar Airways')]
len(qatar_rows)

0

## 3. Bangkok

This dataframe will be used to model and visualize our trends

In [21]:
df_bangkok = read_csv_files("Bangkok", dates_final)
df_bangkok = transfomed_df2(df_bangkok)

In [22]:
df_bangkok = df_bangkok.loc[df_bangkok['out_airline_company'] == df_bangkok['in_airline_company']]
df_bangkok = df_bangkok.drop_duplicates()
df_bangkok = df_bangkok.dropna()
df_bangkok = df_bangkok.reset_index(drop=True)
df_bangkok

,out_airline_company,in_airline_company,dep_city,arr_city,out_dep_date,out_dep_time,out_duration,out_stop_num,out_arr_date,out_arr_time,in_dep_date,in_dep_time,in_duration,in_stop_num,in_arr_date,in_arr_time,hour_scrap,day_scrap,ticket_price,currency
0,Qatar Airways,Qatar Airways,BRU,BKK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,20:00,0 days 23:10:00,1,2023-08-16,14:10,9,20,1368.04,€
1,Qatar Airways,Qatar Airways,BRU,BKK,2023-08-01,09:05,0 days 17:00:00,1,2023-08-02,07:05,2023-08-15,20:00,0 days 23:10:00,1,2023-08-16,14:10,9,20,1390.65,€
2,Qatar Airways,Qatar Airways,BRU,BKK,2023-08-01,09:05,0 days 17:35:00,1,2023-08-02,07:40,2023-08-15,08:45,1 days 10:25:00,1,2023-08-16,14:10,9,20,1290.26,€
3,Qatar Airways,Qatar Airways,BRU,BKK,2023-08-01,09:05,0 days 17:00:00,1,2023-08-02,07:05,2023-08-15,08:45,1 days 10:25:00,1,2023-08-16,14:10,9,20,1312.36,€
4,Lufthansa,Lufthansa,BRU,BKK,2023-08-01,19:00,0 days 14:40:00,2,2023-08-02,14:40,2023-08-15,23:00,0 days 14:10:00,1,2023-08-16,08:10,9,20,1987.74,€
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Lufthansa,Lufthansa,BRU,BKK,2023-08-01,14:40,0 days 19:00:00,2,2023-08-02,14:40,2023-08-15,23:00,0 days 18:25:00,2,2023-08-16,12:25,12,20,2048.36,€
109,Lufthansa,Lufthansa,BRU,BKK,2023-08-01,16:30,0 days 16:40:00,1,2023-08-02,14:10,2023-08-15,23:00,0 days 19:55:00,2,2023-08-16,13:55,12,20,2194.89,€
110,Lufthansa,Lufthansa,BRU,BKK,2023-08-01,16:30,0 days 16:40:00,1,2023-08-02,14:10,2023-08-15,23:00,0 days 19:55:00,2,2023-08-16,13:55,12,20,2048.36,€
111,Lufthansa,Lufthansa,BRU,BKK,2023-08-01,14:40,0 days 18:30:00,2,2023-08-02,14:10,2023-08-15,23:00,0 days 19:00:00,1,2023-08-16,13:00,12,20,2194.89,€
